In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio as i
import math
import cv2
import glob
import os

# Load images from the folders
def load_images_from_folder(folder, tag, flag):
  images = []
  filenames = []
  tags = {}

  # Read each image from the training and testing images folder
  for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
        images.append(img)
        filenames.append(filename)
        # If folder is not testing folder, then tag each image as 0 or 1 based on tag input
        if tag != -1:
          tags[filename] = tag

  # Function call to do the operations on each image
  imageHOGs = operation(images, filenames, flag)

  # If folder is training images then return HOG Descriptors and File Names
  if tag == -1:
    return imageHOGs, filenames
  # Else HOG Descriptors and dictionary of tags with image name as key and tag as value
  else:
    return imageHOGs, tags

# Operations on each image - Convert to grayscale, compute gradient magnitude, gradient angle and HOG
def operation(images, filenames, flag):

  # list to store HOG Descriptors of each image
  imageHOGs = []

  # variable to index list of filenames
  count = 0

  for image in images:

    # Step 1 : Convert RGB image to Grayscale
    gray_image = rgb2gray(image)

    # Step 2 : Prewitt Operation
    gradient_x, gradient_y, gradient_magnitude, gradient_angle = prewitt_operation(gray_image)

    # if image is test image write its gradient magnitude image
    if flag == True:
      cv2.imwrite(str(filenames[count]) + '_gradient_magnitude.jpg', gradient_magnitude)

    # Step 3 : Compute HOG of image
    imageHOG = HOG(gradient_magnitude, gradient_angle)

    # Write the HOG Descriptor to a text file for the specified file names
    if filenames[count] == "crop001028a.bmp" or filenames[count] == "crop001030c.bmp" or filenames[count] == "00000091a_cut.bmp" or filenames[count] == "crop001278a.bmp" or filenames[count] == "crop001500b.bmp" or filenames[count] == "00000090a_cut.bmp":
      with open(str(filenames[count]) + '.txt', 'x') as the_file:
        for item in imageHOG:
          the_file.write(str(item) + "\n")

    imageHOGs.append(imageHOG)
    count += 1

  return imageHOGs


# Step 1 : Convert RBG image to Grayscale
def rgb2gray(rgb):

  r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
  gray = np.round(0.2999 * r + 0.587 * g + 0.114 * b)

  return gray

# Convolution operation
def convolution(mask, image, i, j, centerI, centerJ):

	mask_i, mask_j = 0, 0
	i = i - centerI
	j = j - centerJ
	flag = j
	sum = 0

	while mask_i < len(mask):
		mask_j = 0
		j = flag
		while mask_j < len(mask[0]):
			sum = sum + (mask[mask_i][mask_j] * image[i][j])
			j = j + 1
			mask_j = mask_j + 1
		mask_i = mask_i + 1
		i = i + 1

	return sum

# Step 2 : Prewitt Operation 
def prewitt_operation(image):

  # prewitt operator x
  prewitt_x = [[-1,0,1],
              [-1,0,1],
              [-1,0,1]]

  # prewitt operator y
  prewitt_y = [[1,1,1],
              [0,0,0],
              [-1,-1,-1]]

  height, width = image.shape
  gradient_x = np.zeros((height, width))
  gradient_y = np.zeros((height, width))
  gradient_magnitude = np.zeros((height, width))
  gradient_angle = np.zeros((height, width))

  centerX, centerY = 4, 4
  i = centerX
  j = centerY

  # calculate gradient_x, gradient_y, gradient magnitude and gradient angle for each pixel location
  while i < height - centerX:
    j = centerY
    while j < width - centerY:

      # apply prewitt operator x
      gradient_x[i][j] = convolution(prewitt_x, image, i, j, len(prewitt_x) // 2, len(prewitt_x) // 2)
      
      # absolute value
      gradient_x[i][j] = abs(gradient_x[i][j])

      # normalization step
      gradient_x[i][j] = gradient_x[i][j] / 3

      # apply prewitt operator y
      gradient_y[i][j] = convolution(prewitt_y, image, i, j, len(prewitt_y) // 2, len(prewitt_y) // 2)
      
      # absolute value
      gradient_y[i][j] = abs(gradient_y[i][j])

      # normalization step
      gradient_y[i][j] = gradient_y[i][j] / 3

      gradient_magnitude[i][j] = np.sqrt(gradient_x[i][j]**2  + gradient_y[i][j]**2)

      # normalization step
      gradient_magnitude[i][j] = gradient_magnitude[i][j] / np.sqrt(2)

      if gradient_x[i][j] == 0:
        if gradient_y[i][j] > 0:
          gradient_angle[i][j] = 90
        else:
          gradient_angle[i][j] = -90
      else:
        gradient_angle[i][j] = math.degrees(math.atan((gradient_y[i][j] / gradient_x[i][j])))

      if gradient_angle[i][j] < 0:
        gradient_angle[i][j] += 180

      if gradient_angle[i][j] > 180:
        gradient_angle[i][j] -= 180
      
      if gradient_x[i][j] == 0 and gradient_y[i][j] == 0:
        gradient_magnitude[i][j] = 0
        gradient_angle[i][j] = 0

      j = j + 1
    i = i + 1

  return gradient_x, gradient_y, gradient_magnitude, gradient_angle

# Step 3 : Compute HOG Descriptor of the image
def HOG(gradient_magnitude, gradient_angle):

		# find histogram for each cell of the image
		cellHistogram = computeCellHistogram(gradient_magnitude, gradient_angle)
		cellHistogramSquared = np.square(cellHistogram)

		height, width = gradient_magnitude.shape
		row, col = 0, 0
		numberOfRows = round(height/8)
		numberOfCells = round(width/8)
		HOGDescriptor = np.array([])

		# create final HOG Descriptor
		while row < numberOfRows - 1:
			col = 0
			while col < numberOfCells - 1:

				block = np.array([])
				temp = np.array([])
				block = np.append(block,cellHistogram[row,col])
				block = np.append(block,cellHistogram[row,col+1])
				block = np.append(block,cellHistogram[row+1,col])
				block = np.append(block,cellHistogram[row+1,col+1])
				temp = np.append(temp,cellHistogramSquared[row,col])
				temp = np.append(temp,cellHistogramSquared[row,col+1])
				temp = np.append(temp,cellHistogramSquared[row+1,col])
				temp = np.append(temp,cellHistogramSquared[row+1,col+1])
				temp = np.sum(temp)
    
        # normalization step
				if temp > 0:
					norm = np.sqrt(temp)
					block = (1/norm)*block
				HOGDescriptor = np.append(HOGDescriptor, block)
				col = col + 1
			row = row + 1

		return HOGDescriptor

# Compute histogram for each cell
def computeCellHistogram(gradient_magnitude, gradient_angle):

  #cell size
  cellSize = 8
  height, width = gradient_magnitude.shape

  # number of rows and columns
  numberOfRows = round(height/cellSize)
  numberOfCols = round(width/cellSize)

  # list to store histogram for each cell
  cellHistogram = np.zeros((numberOfRows,numberOfCols,9))

  i, j = 0, 0
  while i < numberOfRows - 1:
    j = 0
    while j < numberOfCols - 1:
      cellHistogram[i, j] = createHistogram(gradient_magnitude, gradient_angle, i, j, cellSize)
      j = j + 1
    i = i + 1
  
  return cellHistogram

# Create histogram for each cell
def createHistogram(gradient_magnitude, gradient_angle, centerI, centerJ, cellSize):

  i, j = centerI * 8, centerJ * 8
  cellHistogram = [0] * 9

  centers = {10 : 0, 30 : 1, 50 : 2, 70 : 3, 90 : 4, 110 : 5, 130 : 6, 150 : 7, 170 : 8}

  while i < centerI * 8 + cellSize:
    j = centerJ * 8
    while j < centerJ * 8 + cellSize:

      angle = gradient_angle[i][j]
      magnitude = gradient_magnitude[i][j]

      # add magnitude to respective histogram bin
      if angle < 10:
        ratio = (10 - angle) / 20
        cellHistogram[8] += ratio * magnitude
        cellHistogram[0] += (1 - ratio) * magnitude
      elif angle > 170:
        ratio = (angle - 160) / 20
        cellHistogram[0] += ratio * magnitude
        cellHistogram[8] += (1 - ratio) * magnitude
      elif angle == 0 or angle == 180:
        cellHistogram[0] += magnitude / 2
        cellHistogram[8] += magnitude / 2
      elif angle in centers.keys():
        cellHistogram[centers.get(angle)] += magnitude
      else:
        index_1 = int(angle // 20) - 1
        index_2 = int(angle // 20 + 1) - 1
        ratio = (index_2 * 20 - angle) / 20
        value_1 = ratio * magnitude
        value_2 = (1 - ratio) * magnitude
        cellHistogram[index_1] += value_1
        cellHistogram[index_2] += value_2
      j = j + 1
    i = i + 1

  return cellHistogram

# Step 4 : Calculate histogram intersection distance
# Compute the distance of the test images with each of the training images using the intersection formula
def histogram_Intersection(trainHOG, tags, testHOG, testFilenames):

  # dictionary to store name of testing image as key and dictionary with name of training image as key and intersection distance as value
  distance_maps = {}

  # convert dictionary keys to list for easier access using indexing
  tags_list = list(tags)

  # find length of each feature vector to use as range in for loop
  limit = len(testHOG[1])

  for i in range(len(testHOG)):
    # empty dictionary to store name of training image as key and distance with testing image as value
    map = {}

    for j in range(len(trainHOG)):
      numerator = 0
      denominator = 0

      for x in range(limit):
        numerator += min(testHOG[i][x], trainHOG[j][x]) # sum of the minimum of testHOG and trainHOG for each of the 7524 vector indices
        denominator += trainHOG[j][x]                   # sum of trainHOG for each of the 7524 vector indices

      map[tags_list[j]] = numerator / denominator       # store the division result in the dictionary

      # Sort the dictionary based on descending order of the values : which is the distance to each of the training images
      sorted_map = sorted(map.items(), key=lambda kv: kv[1], reverse = True)

      # Convert list to dictionary
      sorted_dict = {k : v for k, v in sorted_map}

    # store the sorted map in the final distance map with name of testing image as key
    distance_maps[testFilenames[i]] = sorted_dict

  return distance_maps

# Step 5 : Classify images based on distance with k nearest neighbors
# Classify the images based on the calculated distances to k nearest neighbors
def KNN_Classification(distance_maps, tags, testFilenames, k):

  # Dicitonary with key as testing image name, and value as dictionary with key as training image name and value as intersection distance
  classified_results = {}

  # Iterate through keys of dictionary with key as name testing image
  for key in distance_maps:
    # variable to count tags of neighbors
    human = 0
    no_human = 0

    # variable to break loop after comparing k nearest neighbors
    count = 1

    #Iterate through keys of inner map with key as name of training image and value as intersection distance between testing image HOG and training image HOG
    for inner_key in distance_maps.get(key):
      if(count <= k):
        # tag 1 : Human, 0 : No - Human
        if(tags.get(inner_key) == 1):
          human += 1
        else:
          no_human += 1
        print('Distance of ' + str(key) + ' from ' + str(count) + ' nearest neighbor ' + str(inner_key) + ' whose tag is ' + str(tags.get(inner_key)) + ' is : ' + str(distance_maps.get(key).get(inner_key)))

      # break loop if k nearest neighbors are considered
      else:
        break

      count += 1
        
    # Classify image to have human if human count is greater than no_human count out of k nearest neighbors
    if(human > no_human):
      classified_results[key] = 1
      print('Classification from ' + str(k) + '-NN for ' + str(key) + ' is : Human \n')
    # Classify image as No - Human if human count is less than no_human count out of k nearest neighbors
    else:
      classified_results[key] = 0
      print('Classification from ' + str(k) + '-NN for ' + str(key) + ' is : No - Human \n')

  # Return the dictionary of classified results with key as testing image name and value as classification i.e 1 : Human and 0 : Non - Human
  return classified_results

# Main function call for all processing
def main():

  # Path of folders containing training images
  trainPositive_folder = '/content/Train Positive'
  trainNegative_folder = '/content/Train Negative'

  # Path of folders containing testing images
  testPositive_folder = '/content/Test Positive'
  testNegative_folder = '/content/Test Negative'
  
  # Function call to compute the HOG Descriptor for each image and tag them based on training data (1 : Human, 0 : No - Human)
  trainPositive_imageHOG, positiveTags = load_images_from_folder(trainPositive_folder, 1, False)
  trainNegative_imageHOG, negativeTags = load_images_from_folder(trainNegative_folder, 0, False)

  # Concatenate all training image HOGs to a single list
  trainHOG = trainPositive_imageHOG + trainNegative_imageHOG

  # Concatenate all tags to a single dictionary
  tags = positiveTags.copy()
  tags.update(negativeTags)

  # Function call to compute the HOG Descriptor for each image and write the Gradient Magnitude image 
  testPositive_imageHOG, positiveFilenames = load_images_from_folder(testPositive_folder, -1, True)
  testNegative_imageHOG, negativeFilenames = load_images_from_folder(testNegative_folder, -1, True)

  # Concatenate all testing image HOGs to a single list
  testHOG = testPositive_imageHOG + testNegative_imageHOG
  testFilenames = positiveFilenames + negativeFilenames

  # Step 4 : Calculate histogram intersection distance
  # Function call to find the intersection distance of each Testing HOG to each of the Training HOG and returns a dictionary of dictionaries with key as testing image name and value as dictionary with key as training image name and value as intersection distance
  distance_maps = histogram_Intersection(trainHOG, tags, testHOG, testFilenames)

  # Step 5 : Classify images based on distance with k nearest neighbors
  # Function call to classify the testing images as Human or No - Human based on distance and tags to k Nearest Neighbors
  k = 3 # initialize k to 3 to consider 3 nearest neighbors
  classified_results = KNN_Classification(distance_maps, tags, testFilenames, k)

if __name__ == "__main__":
  main()

Distance of crop001500b.bmp from 1 nearest neighbor crop001672b.bmp whose tag is 1 is : 0.4276689788681655
Distance of crop001500b.bmp from 2 nearest neighbor no_person__no_bike_247_cut.bmp whose tag is 0 is : 0.4214729448336979
Distance of crop001500b.bmp from 3 nearest neighbor crop001030c.bmp whose tag is 1 is : 0.4185015392233208
Classification from 3-NN for crop001500b.bmp is : Human 

Distance of crop001278a.bmp from 1 nearest neighbor crop001008b.bmp whose tag is 1 is : 0.4806977369364223
Distance of crop001278a.bmp from 2 nearest neighbor crop001275b.bmp whose tag is 1 is : 0.46955901206925804
Distance of crop001278a.bmp from 3 nearest neighbor crop001672b.bmp whose tag is 1 is : 0.46170498043617897
Classification from 3-NN for crop001278a.bmp is : Human 

Distance of crop001070a.bmp from 1 nearest neighbor crop001672b.bmp whose tag is 1 is : 0.36251004666057285
Distance of crop001070a.bmp from 2 nearest neighbor no_person__no_bike_259_cut.bmp whose tag is 0 is : 0.359109196976